In [15]:
import pandas as pd
from mysql_connection import *
from transform_API import get_categories
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import requests
import os
from dotenv import load_dotenv 
import sys
import random
from mysql_connection import *
import warnings
warnings.filterwarnings('ignore')

In [4]:
def yelp_ER():
    
    """
        Esta funcion realiza el proceso de ETL completo respecto de la API de yelp para los restaurantes en la base datos mysql.
        Para esto aplica las funciones:
            * extract_businesses
            * transform_business
            * get_table
            * mysql_get_connection
            * get_categories
    
    """
    
    yelp_new_data = pd.read_parquet(r'datalake\business_trasnform.parquet') # Realizo las trasnformaciones necesarias para que los datos esten limpios

    yelp_origen = get_table('business_yelp') # Cargo de la base de datos la tabla de yelp en un dataframe
    
    yelp_new_data = yelp_new_data[~(yelp_new_data['business_id'].isin(yelp_origen['business_id']))] #De los restaurantes extraidos tomo solo los que su id NO esta en la DB
    print(f'La cantidad de restaurantes a ingestar es {yelp_new_data.shape[0]}')
    if yelp_new_data.shape[0] != 0:
        conexion = get_connection_mysql() # Genero una conexion a mysql
        cursor = conexion.cursor() 
        
        consulta = "INSERT INTO business_yelp  VALUES(%s,%s,%s,%s,%s,%s)" 
        yelp_insert = yelp_new_data[['business_id','name','latitude','longitude','stars','state_id']].copy()
        cursor.executemany(consulta,yelp_insert.values.tolist() ) # Inserto los nuevos locales, sin insertar las categorias
        print(f'{yelp_insert.shape[0]} nuevos negocios')
        conexion.commit()
        conexion.close()
        
        categories_origen = get_table('categories') # Cargo la tabla de categorias de la base de datos.    
        max_id = categories_origen['categories_id'].max()
        categorias_new_data = get_categories(yelp_new_data.copy())

        #Agrego la categoria Restaurants a cada local
        
        df_restaurant = categorias_new_data.drop_duplicates(subset='business_id').copy()
        df_restaurants = categorias_new_data.drop_duplicates(subset='business_id').copy()
        
        df_restaurant['categories'] = 'restaurants'
        df_restaurants['categories'] = 'restaurants'
        categorias_new_data = pd.concat([categorias_new_data,df_restaurant])
        categorias_new_data = pd.concat([categorias_new_data,df_restaurants])

        
        
        categorias_new = categorias_new_data[~(categorias_new_data['categories'].isin(categories_origen['name']))] # Selecciono las categorias que no estan en la DB
        categorias_new.loc[:, 'categories_id'] = range(max_id + 1, max_id + 1 + categorias_new.shape[0])


        categories = categorias_new.drop_duplicates(subset='categories').copy() # Elimino las categorias duplicadas y las convierto en lista de listas.
        conexion = get_connection_mysql() 
        cursor = conexion.cursor()
        
        # Ingesto las nuevas categorias.
        consulta = "INSERT INTO categories  VALUES(%s,%s)"
        cursor.executemany(consulta, categories[['categories_id','categories']].values.tolist())
        print(f'{categories.shape[0]} nuevas categorias ingestadas')
        conexion.commit()
        conexion.close()
        
        
        categories_acualizada = get_table('categories') # Cargo la tabla de categorias actualizada.
        
        #Hago un join entre la tabla business_id,categoria creada anteriormente con las categorias de la BD, y me quedo solo con business_id y categoria id
        categorias_yelp_new =  pd.merge(categories_acualizada,categorias_new_data,left_on='name',right_on='categories',how='inner')
        
        conexion = get_connection_mysql()
        
        categorias_yelp_new['categories_id'] = categorias_yelp_new['categories_id'].astype(int)
        
        print(categorias_yelp_new)
        # Como business id ya es unico simplemente agrego las filas a la tabla cateogires_yelp
        try:
            cursor = conexion.cursor()
            consulta = "INSERT INTO categories_yelp  VALUES(%s,%s)"
            cursor.executemany(consulta, categorias_yelp_new[['business_id','categories_id']].values.tolist())
            conexion.commit()
            conexion.close()
        except Exception as e:
            print(f"Error al ejecutar la consulta SQL: {e}")
            # Aquí puedes agregar código adicional para manejar la excepción según tus necesidades.
            # Por ejemplo, podrías hacer un rollback si es necesario.
        finally:
            # Este bloque se ejecutará siempre, asegurando que la conexión se cierre incluso en caso de excepción.
            if conexion and conexion.open:
                conexion.rollback()  # Hacer un rollback en caso de excepción antes de cerrar la conexión.
                conexion.close()
    else:
        return 'No habian restaurantes para ingestar'



    

    



In [5]:
yelp_ER()

La cantidad de restaurantes a ingestar es 0


'No habian restaurantes para ingestar'

In [ ]:
#------------------------------------------------------------------------      
#----------------------------REVIEWS-------------------------------------    
#------------------------------------------------------------------------ 

In [13]:

def yelp_review_ER():
    """
    Esta función realiza el proceso de ETL completo respecto de la API de Yelp para las reviews de restaurantes y las sube en la base de datos MySQL.
    Para esto aplica las funciones:
        * extract_businesses
        * transform_business
        * get_table
        * mysql_get_connection
    """
    # Leer datos de reviews desde el archivo parquet
    
    # Obtener las reviews existentes en la base de datos
    
    review_new_data = pd.read_parquet(r'datalake\reviews_yelp_transform.parquet') # Hago las trasnformaciones sobre el dataframe.
    
    reviews_yelp_origen = get_review_yelp('reviews_yelp') # Consulto la tabla las ultimas resenas

    users_old = get_filtered_table('user_yelp', review_new_data['user_id'].unique().tolist())
    
    #Filtro solo las reviews donde su columna date sea mayor a la maxima existente en la base de datos.
    print(f'{review_new_data.shape[0]} reviews a ingestar')
    
    review_new_data = pd.merge(review_new_data, reviews_yelp_origen[['review_id']], on='review_id', how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
    
    #review_new_data = review_new_data[(~review_new_data['review_id'].isin(reviews_yelp_origen['review_id']))]
    review_new_data['date'] = pd.to_datetime(review_new_data['date'])


    
    #### USERS ####  

    grouped_data = review_new_data.groupby('user_id') # Rrealizo el agrupamiento por usuarios y calculas las metricas.
    min_dates = grouped_data['date'].min()
    first_names = grouped_data['name'].first()
    review_counts = grouped_data['review_id'].count()
    mean_stars = grouped_data['stars'].mean()
    users = pd.DataFrame({
        'name': first_names,
        'creation': min_dates,
        'review_count': review_counts,
        'stars': mean_stars
    }).reset_index()
    users['influence'] = 0
    users = users[['user_id','name','creation','review_count','influence','stars']]
    #### DATAFRAME CON LOS USUARIOS NUEVOS
    new_users = users[~(users['user_id'].isin(users_old['user_id']))]
    new_users['influence'] = 0
    
    print(users.head())
    
     #### DATAFRAME CON LOS USUARIOS EXISTENTES
    #exist_user = users[(users['user_id'].isin(users_old['user_id']))] # Usuarios existentes
    
    
    exist_user = pd.concat([users_old, users[users['user_id'].isin(users_old['user_id'])]]) # Hago un merge de los usuarios de la BD y la llegada
    # Para los nuevos usuarios encuentro la nueva review_count y stars
    exist_user['stars'] = exist_user['stars'].astype('float')
    # Realizo el agrupamiento por usuarios existentes y calculo las metricas.
    grouped_data = exist_user.groupby('user_id')
    min_dates = grouped_data['creation'].min()
    first_names = grouped_data['name'].first()
    review_counts = grouped_data['review_count'].count()
    mean_stars = grouped_data['stars'].mean()
    exist_user = pd.DataFrame({
        'name': first_names,
        'creation': min_dates,
        'review_count': review_counts,
        'stars': mean_stars
    }).reset_index()
    
    if not exist_user.empty:
        try:
            conexion = get_connection_mysql()
            cursor = conexion.cursor()
            
            consulta_new_user = (
            'INSERT INTO user_yelp  VALUES(%s,%s,%s,%s,%s,%s)'
                )

            # Ejecutar la consulta con execumany.
            cursor.executemany(consulta_new_user, new_users[['user_id', 'name','creation', 'review_count', 'influence','stars']].values.tolist())
            print(f'{new_users.shape[0]} usuarios nuevos cargados.')
            # Consulta de actualización con placeholders.
            consulta_old_user = (
                "UPDATE user_yelp "
                "SET name = %s, creation = %s, review_count = %s, stars = %s "
                "WHERE user_id = %s"
            )

            # Ejecutar la consulta con execumany.
            cursor.executemany(consulta_old_user, exist_user[['name', 'creation', 'review_count', 'stars', 'user_id']].values.tolist())
            print(f'{exist_user.shape[0]} usuarios actualizados.')
            conexion.commit()
        except Exception as e:
            print(f"Error al ejecutar la consulta SQL: {e}")
            # Maneja la excepción según tus necesidades.
            # Puedes hacer un rollback si es necesario.
        finally:
            # Cierra la conexión en el bloque finally para asegurar que se cierre incluso en caso de excepción.
            if conexion and conexion.open:
                conexion.rollback()
                conexion.close()

    
    # Verificación de review_id antes de la inserción
    conexion = get_connection_mysql()
    cursor = conexion.cursor()

    review_id_list = review_new_data['review_id'].tolist()

    # Crear una cadena con los review_id para la consulta SQL
    review_id_str = ','.join([f"'{review_id}'" for review_id in review_id_list])

    # Consulta SQL para verificar si los review_id existen en la tabla
    consulta_existencia = f"SELECT review_id FROM reviews_yelp WHERE review_id IN ({review_id_str})"
    cursor.execute(consulta_existencia)

    # Obtener los review_id que existen en la base de datos
    review_id_existente = [row[0] for row in cursor.fetchall()]

    # Filtrar la lista original de review_id para obtener aquellos que no existen en la base de datos
    review_id_no_existente = [review_id for review_id in review_id_list if review_id not in review_id_existente]

    conexion.close()

    # Filtrar el DataFrame para mantener solo los registros con review_id que no existen en la base de datos
    review_new_data_filtered = review_new_data[review_new_data['review_id'].isin(review_id_no_existente)]

    # Inserción de registros en la tabla reviews_yelp para los review_id que no existen
    try:
        conexion = get_connection_mysql()
        cursor = conexion.cursor()

        review_new_data_filtered.drop_duplicates(subset='review_id', inplace=True)

        consulta = "INSERT INTO reviews_yelp VALUES(%s, %s, %s, %s, %s, %s)"
        cursor.executemany(consulta, review_new_data_filtered.drop(columns=['name']).values.tolist())

        
        
        print(f'{review_new_data_filtered.shape[0]} nuevas reviews insertadas')
        print(review_new_data_filtered.head())
        conexion.commit()
        
        return review_new_data_filtered
    except Exception as e:
        print(f"Error al ejecutar la consulta SQL: {e}")
        if conexion:
            conexion.rollback()
    finally:
        if conexion:
            conexion.close()

            

In [14]:
yelp_review_ER()

33 reviews a ingestar
                  user_id        name            creation  review_count  \
0  2b2J7kW1nTVTtiZtizw-tw   Lauren C. 2018-06-29 15:00:02             1   
1  5R3m-2idPFpYq0oRwCK0TQ  Krystle M. 2016-08-06 09:17:25             1   
2  A83xfSiZbdZ_HO00BD7aOw  Bahruny C. 2021-12-06 20:58:25             1   
3  CjPAo8v2K6GrRypugNssMQ    Ticie B. 2018-03-18 01:30:32             1   
4  TkFfkUhNLczx5LB-uR9qaw     Tina S. 2015-11-28 05:28:29             1   

   influence  stars  
0          0    1.0  
1          0    5.0  
2          0    5.0  
3          0    1.0  
4          0    5.0  
0 usuarios nuevos cargados.
33 usuarios actualizados.
0 nuevas reviews insertadas
Empty DataFrame
Columns: [review_id, user_id, business_id, stars, sentiment, date, name]
Index: []


,review_id,user_id,business_id,stars,sentiment,date,name


# Api yelp

In [20]:

api_key_yelp =  "5DIPr_h-AqFcwZuvZboWCR110mmQ6lXWLHYZKpkIiL_QKqguUurch2AwJwkmDk3ywshaUd_MXkYFf0-MG7i1hEd1hIi_8fSGeRU5M0d_Nuc4QwC7NP-rioH0O-lpZXYx" # Cargo la variable de entorno


# Funcion que consulta la API de yelp para obtener los locales por estado.    
def get_business_API(state):
    """Esta funcion realiza una consulta a la API de yelp para obtener los restaurantes por estado.

    Args:
        state (string): Codigo de estado (Texas=TX) del estado requerido. 

    Returns:
        pd.DataFrame: Data frame de la información de un restaurant en concreto.
    """
    
    
    url = f'https://api.yelp.com/v3/businesses/search'

    params = {
        'location': state,
        'categories':','.join(['restaurant','Restaurant','restaurants','Restaurants']),
        'limit':50
    }

    headers = {
        'Authorization': f'Bearer {api_key_yelp}',
        'accept': 'application/json'
    }

    response = requests.get(url, headers=headers,params=params)

    if response.status_code == 200:
        data = response.json()
        # Convierte el JSON en un DataFrame de pandas
        #df = pd.json_normalize(data)
        businesses = pd.json_normalize(data['businesses'])
        return businesses
    else:
        print(f'Error en la solicitud. Código de estado: {response.status_code}')
        print(response.text)

In [23]:
# Funcion que carga desde la API hasta  50 datos de cada estado.        
def extract_businesses():
    
    """ Esta funcion aplica la funcion business para 4 estados seleccionados.

    Returns:
        pd.DataFrame: Concatenacion de los restaurantes de los 4 estados.
    """
    
    
    url = f'https://api.yelp.com/v3/businesses/search'

    yelp_bussines = pd.DataFrame()
    for state in ['CA','FL','NJ','IL']:
        businesses = get_business_API(state)
        yelp_bussines = pd.concat([businesses,yelp_bussines])
    yelp_bussines.to_parquet(r'datalake\business_API.parquet')
    return yelp_bussines


In [24]:
business = extract_businesses()

In [25]:
business

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media1.fl.yelpcdn.com/bphoto/ya6gjD...,False,https://www.yelp.com/biz/girl-and-the-goat-chi...,10021,"[{'alias': 'newamerican', 'title': 'New Americ...",4.5,[delivery],...,41.884181,-87.647947,809 W Randolph,,,Chicago,60607,US,IL,"[809 W Randolph, Chicago, IL 60607]"
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media2.fl.yelpcdn.com/bphoto/T_N7I4...,False,https://www.yelp.com/biz/the-purple-pig-chicag...,8343,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.5,"[pickup, delivery]",...,41.890694,-87.624782,444 N Michigan Ave,None,,Chicago,60611,US,IL,"[444 N Michigan Ave, Chicago, IL 60611]"
2,xoi7Cw7FoknAx5p880RtWQ,au-cheval-chicago,Au Cheval,https://s3-media3.fl.yelpcdn.com/bphoto/td7RDA...,False,https://www.yelp.com/biz/au-cheval-chicago?adj...,8704,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"[pickup, delivery]",...,41.884660,-87.647668,800 W Randolph St,,,Chicago,60607,US,IL,"[800 W Randolph St, Chicago, IL 60607]"
3,y8jVBaHh8ntYkVqDXpsi1Q,cafe-ba-ba-reeba-chicago-3,Cafe Ba-Ba-Reeba!,https://s3-media2.fl.yelpcdn.com/bphoto/eEoh87...,False,https://www.yelp.com/biz/cafe-ba-ba-reeba-chic...,4826,"[{'alias': 'spanish', 'title': 'Spanish'}, {'a...",4.5,"[pickup, delivery]",...,41.918984,-87.648687,2024 N Halsted St,,,Chicago,60614,US,IL,"[2024 N Halsted St, Chicago, IL 60614]"
4,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,Aba,https://s3-media1.fl.yelpcdn.com/bphoto/pEnMDw...,False,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1504,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.5,"[pickup, delivery]",...,41.886944,-87.648795,302 N Green St,Fl 3,None,Chicago,60607,US,IL,"[302 N Green St, Fl 3, Chicago, IL 60607]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,gXis25p88klWtI6b3Ua6Nw,tah-deeg-authentic-persian-cuisine-clovis,Tah Deeg Authentic Persian Cuisine,https://s3-media3.fl.yelpcdn.com/bphoto/dGu0Vg...,False,https://www.yelp.com/biz/tah-deeg-authentic-pe...,246,"[{'alias': 'persian', 'title': 'Persian/Irania...",4.5,"[delivery, pickup]",...,36.807037,-119.692677,1250 Shaw Ave,Ste 109,None,Clovis,93612,US,CA,"[1250 Shaw Ave, Ste 109, Clovis, CA 93612]"
46,XMH4GDn3pBeLMGpvaxCa6w,ararat-market-clovis,Ararat Market,https://s3-media2.fl.yelpcdn.com/bphoto/UTQJg5...,False,https://www.yelp.com/biz/ararat-market-clovis?...,363,"[{'alias': 'armenian', 'title': 'Armenian'}, {...",5.0,[delivery],...,36.838990,-119.699360,497 N Clovis Ave,Ste 205,,Clovis,93611,US,CA,"[497 N Clovis Ave, Ste 205, Clovis, CA 93611]"
47,zax7GbinnYMWMtgW9gdZQg,cracked-pepper-bistro-fresno-2,Cracked Pepper Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/k-gX8F...,False,https://www.yelp.com/biz/cracked-pepper-bistro...,549,"[{'alias': 'newamerican', 'title': 'New Americ...",4.0,[],...,36.835575,-119.810516,6737 N Palm Ave,,,Fresno,93704,US,CA,"[6737 N Palm Ave, Fresno, CA 93704]"
48,8D9h9nK5TqQTErx3fz3XiQ,manhattan-restaurant-fresno,Manhattan Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/1RCjCs...,False,https://www.yelp.com/biz/manhattan-restaurant-...,262,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",4.0,[delivery],...,36.822113,-119.825028,1731 W Bullard Ave,Ste 121,,Fresno,93711,US,CA,"[1731 W Bullard Ave, Ste 121, Fresno, CA 93711]"


In [26]:
# Funcion que apartir de la id de un local extrae las reviews
def reviews_yelp_API(business_id):
    
    """ Esta funcion retorna para un business_id la información de reviews en un DataFrame.

    Returns:
        pd.DataFrame: Data Frame de reviews.
    """
    
    
    url =f'https://api.yelp.com/v3/businesses/{business_id}/reviews?limit=50&sort_by=newest'

    headers = {
        'Authorization': f'Bearer {api_key_yelp}',
        'accept': 'application/json'
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        reviews_list = data.get('reviews', [])
        df = pd.json_normalize(reviews_list)
        return  df

    else:
        print(f'Error en la solicitud. Código de estado: {response.status_code}')
        

In [32]:
# Funcion que a partir de los locales ya existentes en DW extrae las reseñas.
def get_reviewsYelp_API():
    
    """ Esta funcion aplica la funcion reviews_yelp_api  para todos los business que estan en la base de datos aplicando get_table(yelp),
    ademas lo hace 496 para tener en cuenta solo las peticione restantes,
    Falta ver como modificar el business_id de input.

    Returns:
        pd.DataFrame: DataFrame de las revies.
    """
    
    
    yelp = get_table('business_yelp') # Obtengo la tabla de restaurantes de la DB
    business_ids_distinct_list = yelp.dropna(subset='business_id')['business_id'].unique().tolist()# Selecciono solo los valores unicos de business_id
    random.shuffle(business_ids_distinct_list)
    reviews_business = pd.DataFrame()
    iter = 0
    for business_id in business_ids_distinct_list:
        if business_id is None: continue
        if iter <= 10:
            iter += 1
            reviews = reviews_yelp_API(business_id)
            reviews['business_id'] = business_id
            reviews_business = pd.concat([reviews,reviews_business])
            
            
        else :
            reviews_business.to_parquet(r'datalake\reviews_yelp.parquet')
            return reviews_business

    reviews_business.to_parquet(r'datalake\reviews_yelp.parquet')
    return reviews_business


In [ ]:
reviews = get_reviewsYelp_API()
reviews

In [37]:
reviews

,id,url,text,rating,time_created,user.id,user.profile_url,user.image_url,user.name,business_id
0,Jbof6_nat6om6P9efaWUZw,https://www.yelp.com/biz/crocodile-restaurant-...,It's so tempting to give this restaurant a 1-s...,5,2023-08-04 13:23:23,Xp5Hj2SGl0eDp3Ine9a_vg,https://www.yelp.com/user_details?userid=Xp5Hj...,https://s3-media3.fl.yelpcdn.com/photo/vPOHgld...,Harriet E.,lNECraPTMtUPDJPjmF0tVA
1,X8srOZJgG3cnNxdZckjSeA,https://www.yelp.com/biz/crocodile-restaurant-...,"The seating? Amazing, the service? Extremely p...",2,2023-09-03 22:25:53,NxkWWC7wcKj6jp82LGFOYw,https://www.yelp.com/user_details?userid=NxkWW...,https://s3-media3.fl.yelpcdn.com/photo/GyNcdzQ...,Faisal K.,lNECraPTMtUPDJPjmF0tVA
2,dslHaz2Mguk-JvN2_ySPFg,https://www.yelp.com/biz/crocodile-restaurant-...,I was quite surprised when I joined some frien...,4,2023-11-05 20:11:02,iubWB0-vNBWR3E_ZPeKPjA,https://www.yelp.com/user_details?userid=iubWB...,https://s3-media4.fl.yelpcdn.com/photo/wdYFWN8...,Lindsay W.,lNECraPTMtUPDJPjmF0tVA
0,RBf_Mi5C85Ep-mV21ANH7A,https://www.yelp.com/biz/pats-pizzeria-woodbur...,New to the neighborhood and been here a few ti...,1,2020-05-08 09:24:50,TJ-89K1b3Pe05X0VQkVVDQ,https://www.yelp.com/user_details?userid=TJ-89...,None,Mike R.,Lkreype3dg1jMdglCVEhcw
1,133S_YtpF3gPScoGeG8nzA,https://www.yelp.com/biz/pats-pizzeria-woodbur...,My husband and I order here all the time cause...,1,2018-03-23 19:29:30,vhUPxAR_mYG8up6_HCllOw,https://www.yelp.com/user_details?userid=vhUPx...,https://s3-media1.fl.yelpcdn.com/photo/ulCLe_u...,Sarah B.,Lkreype3dg1jMdglCVEhcw
2,NCgE7yIaPReOpS1zzsSrvw,https://www.yelp.com/biz/pats-pizzeria-woodbur...,"If I could give 0 stars, I would. We just cal...",1,2018-08-01 09:38:02,-DhBquVPyKuyUku1z4hB1A,https://www.yelp.com/user_details?userid=-DhBq...,https://s3-media1.fl.yelpcdn.com/photo/pnkkpgg...,Kimberly W.,Lkreype3dg1jMdglCVEhcw
0,u1V0WTKD5gutHaWq-6DBTA,https://www.yelp.com/biz/meet-up-chinese-cuisi...,Came here with 6 starving guys. And Meet Up Ch...,5,2023-07-31 20:42:14,k6AtDPGPoAvjySHN8ZTb2A,https://www.yelp.com/user_details?userid=k6AtD...,https://s3-media4.fl.yelpcdn.com/photo/hzIN9qm...,Kevin S.,RiOcLw__HPf_rTOTpYKnHQ
1,_S1dMbAdVnGX27AH55qqpg,https://www.yelp.com/biz/meet-up-chinese-cuisi...,We decided to try this place after reading the...,4,2023-09-23 11:46:39,6UxlvK6KBBUPozMPytOSoA,https://www.yelp.com/user_details?userid=6Uxlv...,https://s3-media4.fl.yelpcdn.com/photo/3o865gz...,Christy C.,RiOcLw__HPf_rTOTpYKnHQ
2,O8udnCjMjeHxbpC6mbBQ1Q,https://www.yelp.com/biz/meet-up-chinese-cuisi...,Nice hidden gem! First time to have lunch here...,4,2023-05-31 14:42:15,S47Re4B1Qcftzc49ic3l8A,https://www.yelp.com/user_details?userid=S47Re...,https://s3-media2.fl.yelpcdn.com/photo/N1J8A05...,Tina S.,RiOcLw__HPf_rTOTpYKnHQ
0,XqT8Dwf6fb4dWfcwi_4Ilw,https://www.yelp.com/biz/mi-tierra-bonita-fair...,This place is not much on ambiance since it's ...,5,2023-10-22 14:59:45,H4hNy3-EqwgKREYvF1VzkQ,https://www.yelp.com/user_details?userid=H4hNy...,None,Claire M.,jDW4w0kICxhWEfgsy0n-YA
